In [4]:
from newspaper import Article
import nltk
import random
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import tkinter
from tkinter import *

#Punkt pakage
nltk.download('punkt', quiet=True)

#Get article, responses are pulled from the provided URL and displayed to the user.
article = Article('https://en.wikipedia.org/wiki/COVID-19_pandemic')
article.download()
article.parse()
article.nlp()
corpus = article.text

#Tokenization
text = corpus
sentence_list = nltk.sent_tokenize(text) # A list of sentences

def index_sort(list_var):
    length = len(list_var)
    list_index = list(range(0, length))

    x = list_var
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
    return list_index

def bot_response(user_input):
    user_input = user_input.lower()
    sentence_list.append(user_input)
    bot_response = ''
    cm = CountVectorizer().fit_transform(sentence_list)
    similarity_scores = cosine_similarity(cm[-1], cm)
    similarity_scores_list = similarity_scores.flatten()
    index = index_sort(similarity_scores_list)
    index = index[1:]
    response_flag = 0

    j = 0
    for i in range(len(index)):
        if similarity_scores_list[index[i]] > 0.0:
            bot_response = bot_response+' '+sentence_list[index[i]]
            response_flag = 1
            j = j+1
        if j > 2:
            break
    if response_flag == 0:
        bot_response = bot_response+' '+"I did not understand that"

    sentence_list.remove(user_input)
    return bot_response


#Responding with random greetings
def greeting_response(text):
   
    text = text.lower()
    #Generic responses can be edited in the arrays below
    bot_greetings = ['Howdy!', 'Hey!', 'Wuddup?', 'Hello', 'Hi']
    user_greetings = ['hi', 'hey', 'hello','greetings', 'sup']

    for word in text.split():
        if word in user_greetings:
            return random.choice(bot_greetings)
        
exit_list = ['exit', 'bye', 'quit','goodbye']


def send():
    user_input = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    #print('Findie: '+bot_response(user_input))
    ChatLog.config(state=NORMAL)
   # ChatLog.insert(END, "You: " + user_input + '\n\n')
    ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    if user_input != '':
        ChatLog.insert(END, "You: " + user_input + '\n\n')
        if user_input.lower() in exit_list:
            ChatLog.insert(END, "Bot: Goodbye!")
        else:
            if greeting_response(user_input) != None:
                ChatLog.insert(END,'Findie: '+greeting_response(user_input))
                ChatLog.insert(END, "\nAsk me your question!\n")
            else:
                res = bot_response(user_input)
                ChatLog.insert(END, "Bot: " + res + '\n\n')

                ChatLog.config(state=DISABLED)
                ChatLog.yview(END)

                
root = Tk()
root.title("FINDIE")
root.geometry("426x515")
root.resizable(width=FALSE, height=FALSE)


instr = Label(root, text="Hey, This is Findie. I can help you find excerpts from the\n link provided. To exit type bye.\n Happy finding!", font='Verdana 11')
instr.grid(row = 1, column = 0)


#Chat window
ChatLog = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)



ChatLog.config(state=DISABLED)

#Binding scrollbar 
scrollbar = Scrollbar(root, command=ChatLog.yview, cursor="arrow")
ChatLog['yscrollcommand'] = scrollbar.set

#Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="11", height=5, bd=0, 
                    bg="#67d6e5", activebackground="#3c9d9b",fg='#ffffff', command= send )

#box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#All components on the screen
scrollbar.place(x=406,y=70, height=386)
ChatLog.place(x=6,y=70, height=386, width=400)
EntryBox.place(x=6, y=466, height=40, width=294)
SendButton.place(x=305, y=466, height=40, width=115)

root.mainloop()
